# Uses interaction to push triples to the brain

Before running, start GraphDB and make sure that there is a sandbox repository

In [1]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

/Users/selbaez/Documents/PhD/CLTL/cltl-chatbots/venv-cltl-chatbots/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/selbaez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import util.driver_util as d_util
import util.capsule_util as c_util
import intentions.talk as talk
import dummies.text_to_triple as ttt

In [3]:
##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "../../data"

### Define the folder where the images and rdf are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, AGENT)

Directory  ../../data/2021-11-03-18:44:04  Created 
Directory  ../../data/2021-11-03-18:44:04/image  Created 


In [4]:
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                                log_dir=log_path,
                                clear_all=True)
replier = LenkaReplier()

2021-11-03 18:44:08,719 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-11-03 18:44:12,818 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents
2021-11-03 18:44:12,818 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted


In [5]:
#### Small sequence to learn name of speaker
initial_prompt = f"{choice(GREETING)} {choice(ASK_NAME)} {HUMAN_NAME}?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

#### Get name from person  
utterance = input('\n')
print(HUMAN_NAME + ": " + utterance)
textSignal = d_util.create_text_signal(scenario, utterance)
scenario.append_signal(textSignal)

HUMAN_NAME = ttt.get_your_name(utterance)
HUMAN_ID = HUMAN_NAME
chat = Chat(HUMAN_ID)

#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = f"{choice(TALK_TO_ME)}"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

#### Get input and loop
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    ###### Getting the next input signals
    utterance = input('\n')
    print(HUMAN_NAME + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    #### Process input and generate reply
    reply = talk.process_text_and_reply(scenario, place_id, location, HUMAN_ID, textSignal, chat, replier, my_brain)
    print(AGENT + ": " + reply)
    textSignal = d_util.create_text_signal(scenario, reply)
    scenario.append_signal(textSignal)


Leolani2: Hey! Who are you? Stranger?



 Selene


Stranger: Selene
2021-11-03 18:44:17,832 -     INFO - cltl.triple_extraction.api.Chat (Selene)            000 - << Start of Chat with Selene >>
Leolani2: Do you have any gossip?



 I like pizza


Selene: I like pizza
2021-11-03 18:44:24,410 -     INFO -               cltl.triple_extraction.api - Started POS tagger
2021-11-03 18:44:24,415 -     INFO -               cltl.triple_extraction.api - Started NER tagger
2021-11-03 18:44:24,421 -     INFO -               cltl.triple_extraction.api - Loaded grammar
2021-11-03 18:44:25,768 -     INFO - cltl.triple_extraction.api.Chat (Selene)            001 -     Selene: "I like pizza"
2021-11-03 18:44:27,183 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: selene_like_pizza [person_->_food])
2021-11-03 18:44:27,273 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: new subject - new object 
2021-11-03 18:44:30,320 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. dislike agent - 15 gaps as object: e.g. know agent
Leolani2: Interesting! I had never heard about pizza before!


...


 I like thing


Selene: I like thing
2021-11-03 18:46:07,046 -     INFO - cltl.triple_extraction.api.Chat (Selene)            007 -     Selene: "I like thing"
2021-11-03 18:46:07,083 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: selene_like_thing [person_->_state])
2021-11-03 18:46:07,096 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: existing subject - new object 
2021-11-03 18:46:08,545 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. be-ancestor-of person - 15 gaps as object: e.g. be-child-of agent
Leolani2: If you don't mind me asking. What types can selene like



 What do I like


Selene: What do I like
2021-11-03 18:46:19,496 -     INFO - cltl.triple_extraction.api.Chat (Selene)            008 -     Selene: "What do I like"
2021-11-03 18:46:19,499 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: Selene_like_? [person_->_])
Leolani2: I may be ignorant, bu tI don't know.I am sorry, I have no idea


KeyboardInterrupt: Interrupted by user

In [ ]:
#scenario.scenario.end = datetime.now().microsecond
scenarioStorage.save_scenario(scenario)

## End of notebook